In [ ]:
%env OPENAI_API_KEY=key-here

In [ ]:
from datetime import datetime
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat, MagenticOneGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from kagent.tools.k8s import (
    GetPods,
    GetServices,
    GetResources,
    GetPodLogs,
    DescribeResource,
    ExecuteCommand,
    GetResourceYAML,
    GetEvents,
    CreateResource,
    PatchResource,
    CheckServiceConnectivity,
)


model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent responsible for taking the users query and breaking it down tasks into logical, sequential steps.
    
    Please follow these instructions:
    1. **Carefully review the task and identify key components** to ensure all necessary steps are included.
    2. **Break down the task into smaller, manageable sub-tasks** that can be assigned to different team members.
    3. **Order and prioritize the sub-tasks** based on dependencies and requirements.
    4. **Assign each sub-task to the appropriate team member** based on their expertise and role.
    5. **Provide clear instructions and success criteria** for each sub-task to ensure accurate completion.
    6. **Verify the results of each sub-task** to confirm successful completion before proceeding to the next step.
    7. **Summarize the findings** and ensure all components of the task have been addressed.

    When assigning tasks, use this format:
    1. <agent> : <specific task with clear success criteria>

    After all tasks have been completed, please summarize the findings and end with "TERMINATE".

    Your team members are:
        k8s_agent: Handles Kubernetes operations and resource verification
    """,
)

k8s_agent = AssistantAgent(
    "k8s_agent",
    description="An agent for k8s operations",
    tools=[
        GetResources(),
        GetPodLogs(),
        GetResourceYAML(),
        DescribeResource(),
        GetEvents(),
        CreateResource(),
        PatchResource(),
        CheckServiceConnectivity(),
    ],
    model_client=model_client,
    system_message="""
You are a kubernetes expert. You are expected to solve Kubernetes related problems.

# GENERAL RULES:
- You have access to several tools you can use to fix problems.
- Try to reproduce the issue before making changes.
- Prioritize troubleshooting steps in this order:
  1. **Pod deployment issues** (check Deployment, ReplicaSet, StatefulSet status)
  2. **Networking issues** (services misconfigured, selectors not matching)
  3. **Storage issues** (PVC not bound, PV unavailable)
  4. **Permission issues** (service account lacking permissions)
- Issues can be related to Kubernetes objects referenced inside other Kubernetes objects.
- **Do NOT create new namespaces**, use the existing ones.
- **Do NOT modify a pod directly.**, always do the change in the corresponding Deployment, ReplicaSet or StatefulSet.
    """,
)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, k8s_agent],
    model_client=model_client,
    termination_condition=termination,
)

task = "When I call the frontend service at http://frontend-v1:8080 I see an error message telling about issues in the 'backend-v1' service. Can you fix this issue?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

In [ ]:
print(team.dump_component().model_dump_json(indent=2))